<a href="https://colab.research.google.com/github/yujiimt/recsys/blob/master/recsys2019/recsys_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import gc
import re
from tqdm import tqdm
import copy
import os
import pickle

PATH = "/content/drive/My Drive/trivagoRecSysChallengeData2019_v2/"

In [ ]:
meta_df = pd.read_csv(PATH + "item_metadata.csv")
print(len(meta_df))
print("-" * 10)
print(meta_df.isnull().sum())
meta_df.head()

927142
----------
item_id       0
properties    0
dtype: int64


,item_id,properties
0,5101,Satellite TV|Golf Course|Airport Shuttle|Cosme...
1,5416,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Tele...
2,5834,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Tele...
3,5910,Satellite TV|Sailing|Cosmetic Mirror|Telephone...
4,6066,Satellite TV|Sailing|Diving|Cosmetic Mirror|Sa...


In [ ]:
train_df = pd.read_csv(PATH + "train.csv")
print(len(train_df))
print("-" * 10)
print(train_df.isnull().sum())
train_df.head()

15932992
----------
user_id                   0
session_id                0
timestamp                 0
step                      0
action_type               0
reference                 0
platform                  0
city                      0
device                    0
current_filters    14779880
impressions        14346406
prices             14346406
dtype: int64


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


In [ ]:
test_df = pd.read_csv(PATH + "test.csv")
print(len(test_df))
print("-" * 10)
print(test_df.isnull().sum())
test_df.head()

3782335
----------
user_id                  0
session_id               0
timestamp                0
step                     0
action_type              0
reference           261731
platform                 0
city                     0
device                   0
current_filters    3439491
impressions        3253556
prices             3253556
dtype: int64


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,004A07DM0IDW,1d688ec168932,1541555614,1,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN
1,004A07DM0IDW,1d688ec168932,1541555614,2,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN
2,004A07DM0IDW,1d688ec168932,1541555696,3,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
3,004A07DM0IDW,1d688ec168932,1541555707,4,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
4,004A07DM0IDW,1d688ec168932,1541555717,5,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...


In [ ]:
sub = pd.read_csv(PATH + "submission_popular.csv")
print(len(sub))
sub.head()

253573


,user_id,session_id,timestamp,step,item_recommendations
0,000324D9BBUC,89643988fdbfb,1541593942,10,924795 106315 1033140 119494 101758 903037 105...
1,0004Q49X39PY,9de47d9a66494,1541641157,1,3505150 3812004 2227896 2292254 3184842 222702...
2,0004Q49X39PY,beea5c27030cb,1541561202,1,4476010 3505150 3812004 2227896 2292254 222702...
3,00071784XQ6B,9617600e1ba7c,1541630328,2,22854 3067559 22721 22713 16121 22772 22727 22...
4,0008BO33KUQ0,2d0e2102ee0dc,1541636411,6,9857656 5849628 655716 1352530 502066 1405084 ...


In [ ]:
def unroll(t):
  idx = []
  suffix = []
  to_append_partial = []

  a_user = t.loc[0].user_id
  a_sess = t.loc[0].session_id
  count = -1
  to_append = []
  for r in tqdm(zip(t.user_id, t.session_id, t.action_type)):
    count += 1
    if r[0] == a_user and r[1] == a_sess:
      to_append.append(count)
      if r[2] == "clickout item":
        to_append_partial += copy.deepcopy([to_append])
      else:
        for i in range(len(to_append_partial) -1):
          idx += to_append_partial[i]
          suffix += ["_"*(i+1) for j in to_append_partial[i]]
        idx += to_append
        suffix += ["" for j in to_append]
        a_user = t.loc[count].user_id
        a_sess = t.loc[count].session_id
        to_append = [count]
        to_append_partial = []
        if r[2] == 'clickout item':
          to_append_partial += copy.deepcopy([to_append])
  for i in range(len(to_append_partial)-1):
      idx += to_append_partial[i]
      suffix += ["_"*(i+1) for j in to_append_partial[i]]
      
  t_new = t.loc[idx]
  user_with_suffix = []
  sess_with_suffix = []
  count = 0
  for r in tqdm(zip(t_new.user_id, t_new.session_id)):
    user_with_suffix.append("{}{}".format(r[0], suffix[count]))
    sess_with_suffix.append("{}{}".format(r[1], suffix[count]))
    count += 1
  t_new["user_id"] = user_with_suffix
  t_new["session_id"] = sess_with_suffix
  return t_new

In [ ]:
def reset_step_for(df):
  res_df = df.copy()
  df_dup = df[["session_id", "user_id", "step"]]
  df_dup = df_dup[df_dup["step"] == 1]
  df_dup = df_dup.groupby(["user_id", "session_id", "step"]).size() \
    .sort_values(ascending=False) \
    .reset_index(name="count")
  df_dup = df_dup[df_dup["count"] > 1]
  df_dup = df_dup[["user_id", "session_id"]]

  for _, row in tqdm(df_dup.iterrows()):
    mask = (df.user_id == row.user_id) & (df.session_id == row.session_id)
    sess_length = sum(mask*1)
    res_df.loc[mask, "step"] = np.arange(1, sess_length+1, dtype="int")

  return res_df 

In [ ]:
def merge_duplicates(df):
    tqdm.pandas()

    duplicates_indices = []
    indices = df.index.values
    totlen = len(df)
    i = 0
    j = 0
    next_index = indices[j]

    for index in tqdm(indices):
        if i >= j:
            curr_actiontype = df.at[index,'action_type']
            count = 1
            j += 1
            while j < totlen:
                next_index = indices[j]

                if curr_actiontype != 'clickout item' and \
                    df.at[index, 'user_id'] == df.at[next_index, 'user_id'] and \
                    df.at[index, 'session_id'] == df.at[next_index, 'session_id'] and \
                    df.at[index, 'reference'] == df.at[next_index, 'reference'] and \
                    curr_actiontype == df.at[next_index, 'action_type']:

                    j += 1
                    duplicates_indices.append(next_index)
                    count += 1
                else:
                    break

            df.at[index, 'frequence'] = count
        i += 1

    return df.drop(duplicates_indices)

In [ ]:
def create_full_df(custom_preprocess_function):
  train_df, test_df = custom_preprocess_function(train_data, reset_index(drop=True),
                                                 test_data, reset_index(drop=True))
  train_df = train_df.reset_index(drop=True)
  test_df = test_df.reset_index(drop=True)

  #test
  len_original_train = train_df.shape[0]
  compressed = menu.yesno_choice(title="Do you want the compressed version? (no for the original full",
                                 callback_yes = lambda: True, callback_no = lambda: False)
  
  print("-"*15)
  train_df = reset_step_for_duplicated_session(train_df)


  #train merging duplicates

  if compressed:
    train_df = merge_duplicates(train_df)
  else:
    train_df["frequence"] = 1

  del train_df


  with open(data, "a", encoding="utf-8") as f:
    test_df.index += len_original_train

    test_df = reset_step_for(test_df)


    if compressed:
      test_df = merge_duplicates(test_df)
    else:
      test_df["frequence"] = 1

    mask = (test_df["action_type"] != "clickout item") & (test_df["reference"].isnull())
    test_df = test_df.drop(test_df[mask].index)

    test_df.to_csv(f, header=False)

In [ ]:
def get_target_indices(df):
  df = df[(df["action_type"] == "clickout item") & (df["reference"].isnull())]
  return df.index.values

In [ ]:
def split(df, save_path, perc_train=80):
  sorted_session_ids = df.groupby("session_id").first().sort_values("timestamp").reset_index()["session_id"]
  print("slicing")
  slice_sorted_session_ids = sorted_session_ids.head(int(len(sorted_session_ids) * (perc_train / 100)))
  df_train = df.loc[df["session_id"].isin(slice_sorted_session_ids)]
  df_test = df.loc[~df["session_id"].isin(slice_sorted_session_ids)]

  groups = df_test[df_test["action_type"] == 'clickout item'].groupby("user_id", as_index=False)
  remove_reference_tuples = groups.apply(lambda x: x.sort_values(by=["timestamp"], ascending=True).tail(1))

  for index, row in tqdm(remove_reference_tuples.iterrows()):
    if int(row["refernce"]) not in list(map(int, row["impressions"].split('|'))):
      remove_reference_tuples.drop(index, inplace=True)
  

  for e in tqdm(remove_reference_tuples.index.tolist()):
    df_test.at[e[1], "reference"] = np.nan

  df_train.to_csv(os.path.join(save_path, "train.csv"))
  df_test.to_csv(os.path.join(save_path, "test.csv"))

  np.save(os.path.join(save_path, 'train_indices'), df_train.index)
  np.save(os.path.join(save_path, 'test_indices'), df_test.index)
  print('Done!')

In [ ]:
def append_missing_accomodations(mode):
  found_ids = []
  joined_df = data.train_df(mode).append(data.test_df(mode))


  refs = joined_df.reference
  refs = refs[refs.notnull()].values

  for r in tqdm(refs):
    try:
      v = int(r)
      found_ids.append(v)
    except ValueError:
      continue
  
  found_ids = set(found_ids)
  acs = data.accomdations_ids()
  accomod_known = set(map(int, acs))
  missing = found_ids.difference(accomod_known)
  missing_count = len(missing)

  del joined_df

  if missing_count > 0:
    new_acc_df = pd.DataFrame({'item_id' : list(missing)},
                              columns = ['item_id', "properties"])
    new_acs = data.accomdations_df().append(new_acc_df, ignore_index=True)
    new_acs.to_csv(ITEMS_PATH, index=False)
    print("complete")

In [ ]:
def preprocess_accomodations_df(preprocessing_fns):
  assert isinstance(preprocessing_fns, list)

  accomodations_df = data.accomodations_original_df()

  tqdm.pandas()

  for preprfnc in preprocessing_fns:
    accomodations_df = accomodations_df.progress_apply(preprfnc, axis=1, result_type='broadcast')

  accomodations_df.to_csv(ITEMS_PATH, index=False)

In [ ]:
def remove_from_satars_features(row):
  propts_to_remove = ['From 2 Stars','From 3 Stars','From 4 Stars']
  if isinstance(row.properties, str):
    propts = row.properties.split('|')
    return row.item_id, '|'.join([p for p in propts if p not in propts_to_remove])
  else:
    return row

In [ ]:
def preprpcess():
  def _create_csvs():
    path = ""
    full = data.full_df()
    train_len = data.read_confing()[data.TRAIN_LEN_KEY]

    train = full.iloc[0:train_len]
    test = full.iloc[train_len:len(full)]
    target_indices = get_target_indices(test)


    check_folder('dataset/preprocessed/no_cluster/full')
    train.to_csv(os.path.join(path, 'full/train.csv'))
    test.to_csv(os.path.join(path, 'full/test.csv'))
    np.save(os.path.join(path, 'full/train_indices'), train.index)
    np.save(os.path.join(path, 'full/test_indices'), test.index)
    np.save(os.path.join(path, 'full/target_indices'), target_indices)

    no_of_rows_in_small = int(input("How many rows do you want in small.csv?"))
    train_small = get_small_dataset(train, maximum_rows = no_of_rows_in_samll)
    
    check_folder('dataset/preprocessed/no_cluster/small')
    split(train_small, os.path.join(path, 'small'))

    check_folder('dataset/preproesssed/no_cluster/local')
    split(train, os.apth.join(path, 'local'))
    
    original_item_metadata = data.accomodations_original_df()
    original_item_metadata
